# Spark SQL

In [1]:
import os
os.environ['SPARK_HOME'] = r"C:\Users\Dani\Documents\Python Scripts\Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ["HADOOP_HOME"] = r"C:\hadoop\hadoop-3.2.2"
os.environ["PATH"] += r";C:\hadoop\hadoop-3.2.2\bin"

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder.appName("SQLSpark").getOrCreate()

In [4]:
df = spark.read.parquet("data/house-price.parquet")
df.printSchema()
df.show(5)

root
 |-- price: long (nullable = true)
 |-- area: long (nullable = true)
 |-- bedrooms: long (nullable = true)
 |-- bathrooms: long (nullable = true)
 |-- stories: long (nullable = true)
 |-- mainroad: string (nullable = true)
 |-- guestroom: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- hotwaterheating: string (nullable = true)
 |-- airconditioning: string (nullable = true)
 |-- parking: long (nullable = true)
 |-- prefarea: string (nullable = true)
 |-- furnishingstatus: string (nullable = true)

+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|   price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|13300000|7420|       4|        2|      3|     yes|       no

In [5]:
df.createOrReplaceTempView("housing_df")

In [6]:
test = spark.sql("SELECT * FROM housing_df ORDER BY area")
test.show(5)

+-------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|  price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+-------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|3150000|1650|       3|        1|      2|      no|       no|     yes|             no|             no|      0|      no|     unfurnished|
|1890000|1700|       3|        1|      2|     yes|       no|      no|             no|             no|      0|      no|     unfurnished|
|2275000|1836|       2|        1|      1|      no|       no|     yes|             no|             no|      0|      no|  semi-furnished|
|4340000|1905|       5|        1|      2|      no|       no|     yes|             no|             no|      0|      no|  semi-furnished|
|3430000|1950|       3|        2|      2|     ye

In [10]:
spark.catalog.tableExists("housing_df")

True

In [29]:
new_df = spark.sql("SELECT area, AVG(price) AS avg_price, SUM(bedrooms+bathrooms+stories) AS total_places, " \
"COUNT(CASE WHEN mainroad = 'yes' THEN area END) AS houses_with_mainroad," \
"SUM(parking) as total_parking," "ARRAY_AGG(DISTINCT furnishingstatus) AS furnishing_statuses FROM housing_df GROUP BY ALL")

new_df.show(20, False)

+----+------------------+------------+--------------------+-------------+-----------------------------+
|area|avg_price         |total_places|houses_with_mainroad|total_parking|furnishing_statuses          |
+----+------------------+------------+--------------------+-------------+-----------------------------+
|1650|3150000.0         |6           |0                   |0            |[unfurnished]                |
|1700|1890000.0         |6           |1                   |0            |[unfurnished]                |
|1836|2275000.0         |4           |0                   |0            |[semi-furnished]             |
|1905|4340000.0         |8           |0                   |0            |[semi-furnished]             |
|1950|2835000.0         |12          |1                   |0            |[unfurnished]                |
|2000|2660000.0         |5           |1                   |0            |[semi-furnished]             |
|2015|3850000.0         |6           |1                   |0    